# <center> <font style="color:rgb(100,109,254)">   Creating a Virtual Pen & Eraser </font> </center>


In [2]:
import cv2
import numpy as np
import time

## <font style="color:rgb(134,19,348)">Step 1: Find Color range of target Pen and save it</font>


In [3]:
# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Intializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 tracbars that will control the lower and upper range of H,S & V channels.
# The Arguments are like this: Name of trackbar, window name, range, callback function.
# For Hue the range is 0-179 and for S,V its 0-255.
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
 
while True:
    
    # Start reading the webcam feed frame by frame.
    ret, frame = cap.read()
    if not ret:
        break
    # Flip the frame horizontally (Not required)
    frame = cv2.flip( frame, 1 ) 
    
    # Convert the BGR image to HSV image.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get the new values of the trackbar in real time as the user changes them
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # Set the lower and upper HSV range according to the value selected by the trackbar
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # Filter the image and get the binary mask, where white represents your target color
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # You can also visualize the real part of the target color (Optional)
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Converting the binary mask to 3 channel image, this is just so we can stack it with the others
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack the mask, orginal frame and the filtered result
    stacked = np.hstack((mask_3,frame,res))
    
    # Show this stacked frame at 40% of the size.
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    # If the user presses ESC then exit the program
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    # If the user presses `s` then print this array.
    if key == ord('s'):
        
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as penval.npy
        np.save('penval',thearray)
        break
    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

## <font style="color:rgb(134,19,348)">Step 2: Maximizing the Detection Mask and Getting rid of the noise   </font>

## <font style="color:rgb(134,19,348)">Step 3: Tracking the Target Pen  </font>

## <font style="color:rgb(134,19,348)">Step 4: Drawing with the Pen   </font>

## <font style="color:rgb(134,19,348)">Step 5: Adding An Image Wiper  </font>


## <font style="color:rgb(134,19,348)">Step 6: Adding the Eraser Functionality</font>


In [3]:
import cv2
import numpy as np
import time

load_from_disk = True
if load_from_disk:
    penval = np.load("penval.npy")
 
cap = cv2.VideoCapture(0)
 
# Load these 2 images and resize them to the same size.
pen_img = cv2.resize(cv2.imread("pen.png",1), (50, 50))
eraser_img = cv2.resize(cv2.imread("eraser.jpg",1), (50, 50))
 
kernel = np.ones((5,5),np.uint8)
 
# Making window size adjustable
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
 
# This is the canvas on which we will draw upon
canvas = None
 
# Create a background subtractor Object
backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows = False)
 
# This threshold determines the amount of disruption in the background.
background_threshold = 600
 
# A variable which tells you if you're using a pen or an eraser.
switch = 'Pen'
 
# With this variable we will monitor the time between previous switch.
last_switch = time.time()
 
# Initilize x1,y1 points
x1,y1=0,0
 
# Threshold for noise
noiseth = 800
 
# Threshold for wiper, the size of the contour must be bigger than this for # us to clear the canvas
wiper_thresh = 40000
 
# A variable which tells when to clear canvas
clear = False
 
while(1):
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )
     
    # Initilize the canvas as a black image
    if canvas is None:
        canvas = np.zeros_like(frame)
         
    # Take the top left of the frame and apply the background subtractor
    # there    
    top_left = frame[0: 50, 0: 50]
    fgmask = backgroundobject.apply(top_left)
     
    # Note the number of pixels that are white, this is the level of 
    # disruption.
    switch_thresh = np.sum(fgmask==255)
     
    # If the disruption is greater than background threshold and there has 
    # been some time after the previous switch then you. can change the 
    # object type.
    if switch_thresh>background_threshold and (time.time()-last_switch) > 1:
 
        # Save the time of the switch. 
        last_switch = time.time()
         
        if switch == 'Pen':
            switch = 'Eraser'
        else:
            switch = 'Pen'
 
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
     
    # If you're reading from memory then load the upper and lower ranges 
    # from there
    if load_from_disk:
            lower_range = penval[0]
            upper_range = penval[1]
             
    # Otherwise define your own custom values for upper and lower range.
    else:             
       lower_range  = np.array([26,80,147])
       upper_range = np.array([81,255,255])
     
    mask = cv2.inRange(hsv, lower_range, upper_range)
     
    # Perform morphological operations to get rid of the noise
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)
     
    # Find Contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, 
    cv2.CHAIN_APPROX_SIMPLE)
     
    # Make sure there is a contour present and also it size is bigger than 
    # noise threshold.
    if contours and cv2.contourArea(max(contours,
                                      key = cv2.contourArea)) > noiseth:
                 
        c = max(contours, key = cv2.contourArea)    
        x2,y2,w,h = cv2.boundingRect(c)
         
        # Get the area of the contour
        area = cv2.contourArea(c)
         
        # If there were no previous points then save the detected x2,y2 
        # coordinates as x1,y1. 
        if x1 == 0 and y1 == 0:
            x1,y1= x2,y2
             
        else:
            if switch == 'Pen':
                # Draw the line on the canvas
                canvas = cv2.line(canvas, (x1,y1),
                (x2,y2), [255,0,0], 5)
                 
            else:
                cv2.circle(canvas, (x2, y2), 20,
                (0,0,0), -1)
             
             
         
        # After the line is drawn the new points become the previous points.
        x1,y1= x2,y2
         
        # Now if the area is greater than the wiper threshold then set the 
        # clear variable to True
        if area > wiper_thresh:
           cv2.putText(canvas,'Clearing Canvas',(0,200), 
           cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 1, cv2.LINE_AA)
           clear = True
 
    else:
        # If there were no contours detected then make x1,y1 = 0
        x1,y1 =0,0
     
    
    # Now this piece of code is just for smooth drawing. (Optional)
    _ , mask = cv2.threshold(cv2.cvtColor (canvas, cv2.COLOR_BGR2GRAY), 20, 
    255, cv2.THRESH_BINARY)
    foreground = cv2.bitwise_and(canvas, canvas, mask = mask)
    background = cv2.bitwise_and(frame, frame,
    mask = cv2.bitwise_not(mask))
    frame = cv2.add(foreground,background)
 
    # Switch the images depending upon what we're using, pen or eraser.
    if switch != 'Pen':
        cv2.circle(frame, (x1, y1), 20, (255,255,255), -1)
        frame[0: 50, 0: 50] = eraser_img
    else:
        frame[0: 50, 0: 50] = pen_img
 
    cv2.imshow('image',frame)
 
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
     
    # Clear the canvas after 1 second, if the clear variable is true
    if clear == True: 
        time.sleep(1)
        canvas = None
         
        # And then set clear to false
        clear = False
         
cv2.destroyAllWindows()
cap.release()


In [11]:
import cv2
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
import time

# Load penval if necessary
load_from_disk = True
if load_from_disk:
    penval = np.load('penval.npy')

# Setup the video capture device
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

# Load images for the pen and eraser
pen_img = cv2.resize(cv2.imread('pen.png', cv2.IMREAD_COLOR), (50, 50))
eraser_img = cv2.resize(cv2.imread('eraser.jpg', cv2.IMREAD_COLOR), (50, 50))

# Create a background subtractor
backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
background_threshold = 600
switch = 'Pen'
last_switch = time.time()
x1, y1 = 0, 0
noiseth = 800
wiper_thresh = 40000
clear = False
kernel = np.ones((5,5),np.uint8)

def show_frame():
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)

    global x1, y1, clear, switch, last_switch

    # Process the frame
    top_left = frame[0:50, 0:50]
    fgmask = backgroundobject.apply(top_left)
    switch_thresh = np.sum(fgmask == 255)

    if switch_thresh > background_threshold and (time.time() - last_switch) > 1:
        last_switch = time.time()
        switch = 'Eraser' if switch == 'Pen' else 'Pen'

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])

    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > noiseth:
        c = max(contours, key=cv2.contourArea)
        x2, y2, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)

        if x1 == 0 and y1 == 0:
            x1, y1 = x2, y2
        else:
            if switch == 'Pen':
                draw_on_canvas(x1, y1, x2, y2, False)
            else:
                draw_on_canvas(x1, y1, x2, y2, True)
        x1, y1 = x2, y2

        if area > wiper_thresh:
            clear = True

    else:
        x1, y1 = 0, 0

    # Overlay pen or eraser icon
    if switch == 'Pen':
        frame[0:50, 0:50] = pen_img
    else:
        frame[0:50, 0:50] = eraser_img

    # Resize frame for display only
    display_frame = cv2.resize(frame, (600, 300))

    # UI Updates for the display frame
    cv2image = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGBA)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    display1.imgtk = imgtk
    display1.configure(image=imgtk)
    if clear:
        clear_canvas()
        clear = False
    root.after(10, show_frame)

def draw_on_canvas(x1, y1, x2, y2, is_eraser):
    if is_eraser:
        drawing_canvas.create_line(x1, y1, x2, y2, fill='white', width=20)
    else:
        drawing_canvas.create_line(x1, y1, x2, y2, fill='black', width=5)

def clear_canvas():
    drawing_canvas.delete("all")

# Setup the main window
root = tk.Tk()
root.bind('<Escape>', lambda e: root.quit())
root.title("Drawing Application")
root.geometry('1920x1080')  # Set a fixed large size if preferred

main_frame = tk.Frame(root)
main_frame.pack(fill=tk.BOTH, expand=True)

# Display video using tk.Label
display1 = tk.Label(main_frame, width=600, height=300)  # Video size
display1.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)  # Top for video

# Setup the drawing canvas (larger size)
drawing_canvas = tk.Canvas(main_frame, bg="white", width=1280, height=720)  # Larger canvas
drawing_canvas.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)  # Bottom for canvas

# Button to clear the canvas
clear_button = tk.Button(root, text="Clear", command=clear_canvas)
clear_button.pack(side=tk.BOTTOM, fill=tk.X)  # Button at the bottom

show_frame()  # Display
root.mainloop()  # Starts GUI

cap.release()
cv2.destroyAllWindows()


In [15]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import Button, Frame
from PIL import Image, ImageTk
import time

# Load calibration file
load_from_disk = True
if load_from_disk:
    penval = np.load("penval.npy")

# Video capture
cap = cv2.VideoCapture(0)

# Kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)

# Background subtractor
backgroundobject = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
background_threshold = 600
switch = 'Pointer'
last_switch = time.time()

# Initialize points
x1, y1 = 0, 0
noiseth = 800
wiper_thresh = 40000
clear = False

# Tkinter setup
root = tk.Tk()
root.title("Finger Drawing App")

# Frame for buttons
button_frame = Frame(root)
button_frame.pack(side=tk.LEFT, fill=tk.Y, padx=10, pady=10)

# Canvas for drawing
canvas = tk.Canvas(root, width=800, height=600, bg='white')
canvas.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)

# Button functions
def set_tool(tool):
    global switch
    switch = tool

def clear_canvas():
    canvas.delete("all")

# Buttons
btn_pointer = Button(button_frame, text="Pointer", command=lambda: set_tool("Pointer"))
btn_pen = Button(button_frame, text="Pen", command=lambda: set_tool("Pen"))
btn_eraser = Button(button_frame, text="Eraser", command=lambda: set_tool("Eraser"))
btn_clear = Button(button_frame, text="Clear Canvas", command=clear_canvas)
btn_pointer.pack(pady=5)
btn_pen.pack(pady=5)
btn_eraser.pack(pady=5)
btn_clear.pack(pady=5)

# Label for displaying frames
frame_label = tk.Label(root)
frame_label.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)

def update_frame():
    global x1, y1, clear, last_switch
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    top_left = frame[0: 50, 0: 50]
    fgmask = backgroundobject.apply(top_left)
    switch_thresh = np.sum(fgmask == 255)
    
    if switch_thresh > background_threshold and (time.time() - last_switch) > 1:
        last_switch = time.time()

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if load_from_disk:
        lower_range = penval[0]
        upper_range = penval[1]
    else:
        lower_range = np.array([26, 80, 147])
        upper_range = np.array([81, 255, 255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > noiseth:
        c = max(contours, key=cv2.contourArea)
        x2, y2, w, h = cv2.boundingRect(c)
        
        if switch == 'Pointer':  # Remove previous pointer indicators
            canvas.create_oval(x2-10, y2-10, x2+10, y2+10, outline="green", width=2, tags="pointer")
            canvas.delete("pointer")
            # canvas.create_oval(x2-5, y2-5, x2+5, y2+5, fill="red", outline="red", tags="pointer")
        elif switch == 'Pen':
            canvas.create_line(x1, y1, x2, y2, fill="blue", width=5)
        elif switch == 'Eraser':
            canvas.create_oval(x2-20, y2-20, x2+20, y2+20, fill="white", outline="red", width=5)
        
        x1, y1 = x2, y2
        
        if cv2.contourArea(c) > wiper_thresh:
            clear = True
    else:
        x1, y1 = 0, 0

    if clear:
        canvas.delete("all")
        clear = False

    frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frame_photo = ImageTk.PhotoImage(image=frame_image)
    frame_label.imgtk = frame_photo
    frame_label.configure(image=frame_photo)
    frame_label.after(10, update_frame)

update_frame()
root.mainloop()

cap.release()
cv2.destroyAllWindows()
